In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from implementations import *
from cross_validation import *
from losses import *
from helpers import *

In [151]:
DATA_TRAIN_PATH = '../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=False)

In [164]:
indices = np.random.permutation(len(y))
train_percentage = 0.8
cutoff_idx = int(train_percentage*len(y))
y_train = y[:cutoff_idx]
tX_train = tX[:cutoff_idx]
y_test = y[cutoff_idx:]
tX_test = tX[cutoff_idx:]

In [165]:
### Relabel the output y from {-1,1} to {0,1}

y_train_0 =y_train.copy()
y_train_0[y_train_0 == -1] = 0


### Dealing with invalid data & Normalize data
# a. Dataset where columns containing invalid data are dropped
tX_train_drop_invalid = tX_train.copy()
tX_train_drop_invalid = tX_train_drop_invalid[:, ~np.any(tX_train_drop_invalid == -999., axis=0)]

tX_train_di_means = np.mean(tX_train_drop_invalid, axis=0)
tX_train_di_stds = np.std(tX_train_drop_invalid, axis=0)

tX_train_drop_invalid = (tX_train_drop_invalid - tX_train_di_means)/tX_train_di_stds

# b. Dataset where columns containing invalid data are replaced by the mean of the corresponding feature
tX_train_replace_invalid = tX_train.copy()
tX_train_replace_invalid[tX_train_replace_invalid == -999.] = np.nan

tX_train_ri_means = np.nanmean(tX_train_replace_invalid, axis=0)
tX_train_ri_stds = np.nanstd(tX_train_replace_invalid, axis=0)

tX_train_replace_invalid = (tX_train_replace_invalid - tX_train_ri_means)/tX_train_ri_stds
tX_train_replace_invalid[np.isnan(tX_train_replace_invalid)] = 0

### Preprocessing for Test set (using the value calculated in training!)
tX_test_drop_invalid = tX_test.copy()
tX_test_drop_invalid = tX_test_drop_invalid[:, ~np.any(tX_test == -999., axis=0)]
tX_test_drop_invalid = (tX_test_drop_invalid - tX_train_di_means)/tX_train_di_stds

tX_test_replace_invalid = tX_test.copy()
tX_test_replace_invalid[tX_test_replace_invalid == -999.] = np.nan
tX_test_replace_invalid = (tX_test_replace_invalid - tX_train_ri_means)/tX_train_ri_stds
tX_test_replace_invalid[np.isnan(tX_test_replace_invalid)] = 0

In [ ]:
def standardize(x):
    mean_x = np.mean(x, axis=0)
    x = x - mean_x
    std_x = np.std(x, axis=0)
    x = x / std_x
    return x, mean_x, std_x

def standardize_test(x, mean, std):
    x = x - mean
    x = x / std
    return x

        


In [166]:
print(tX_train_replace_invalid.shape)
print(tX_train_drop_invalid.shape)

(200000, 30)
(200000, 19)


In [52]:
def get_accuracy(y, tx, w, threshold):
    pred = tx@w
    pred_class = [0 if p<threshold else 1 for p in pred]
    n = len(y)
    correct=0
    for i in range(n):
        if pred_class[i] == y[i]:
            correct+=1
    return correct/n

In [16]:
def lambda_gamma_ridge_cv(y, tx, lambdas, K, max_iters, batch_size, seed):
  """Do K-fold cross-validation with ridge regression for each value in lambdas, at every iteration.
  
  Inputs:
  y : np array
    (N, 1) or (N,)
  tx : np array
    (N, D)
  lambdas : iterable
    Regularisation parameters for cost function
  K : int
    Number of folds
  max_iters : int
    Maxium number of iterations for SGD
  batch_size : int
    Size of mini-batches
  seed : int
    Seed for pseudo-random number generation
  
  Outputs:
  w_best : np array
    (D, len(lambdas))
    Trained weights that produced the smallest validation error
    over all folds, for each lambda and gamma
  training_errors : np array
    (K, len(lambdas))
    Training loss for each fold, for each lambda
  validation_errors : np array
    (K, len(lambdas))
    Validation loss for each fold, for each lambda
  """
  y, tx = prepare_dimensions(y, tx)

  N = len(y)
  len_lambdas = len(lambdas)

  initial_w = np.ones((tx.shape[1], 1))
  w_best = np.zeros((tx.shape[1], len_lambdas))

  training_errors = np.zeros((K, len_lambdas))
  validation_errors = np.zeros((K, len_lambdas))
  min_error = np.inf * np.ones((len_lambdas))

  k_indices = build_k_indices(y, K, seed)

  for k in range(K):
    # Take all but the k-th row of tx and y
    tx_train, y_train = map(lambda a: a[np.delete(k_indices, k).flatten()], (tx, y))
    # Take the k-th row of tx and y
    tx_test, y_test = map(lambda a: a[k_indices[k]], (tx, y))

    for i, lambda_ in enumerate(lambdas):
      # Train
      w, loss_tr = ridge_regression(y, tx, lambda_)
      # Test
      loss_te = compute_mse_loss(y_test, tx_test, w)
      
      training_errors[k, i] = loss_tr
      validation_errors[k, i] = loss_te

      # Keep the weights that give the lowest loss_te
      if loss_te < min_error[i]:
        min_error[i] = loss_te
        w_best[:, i] = w.ravel()

  return w_best, training_errors, validation_errors

In [167]:
lambdas = np.array([0])
w_best, training_errors, validation_errors = lambda_gamma_ridge_cv(y_train_0, tX_train_replace_invalid, lambdas, 4, 1000, 1, 1)
np.mean(validation_errors)

0.1436853280529944

In [168]:
lambdas = np.array([0])
w_best, training_errors, validation_errors = lambda_gamma_ridge_cv(y_train_0, tX_train_drop_invalid, lambdas, 4, 1000, 1, 1)
np.mean(validation_errors)

0.14668950310408446

In [170]:
lambdas = np.logspace(-8, -1, 8)
w_best, training_errors, validation_errors = lambda_gamma_ridge_cv(y_train_0, tX_train_replace_invalid, lambdas, 4, 1000, 1, 1)
val_loss_per_lambda = np.mean(validation_errors, axis=0, dtype=np.float64)
np.argmin(val_loss_per_lambda)
print(val_loss_per_lambda[3])

0.14368542590114566


In [171]:
lambdas = np.logspace(-8, -1, 8)
w_best, training_errors, validation_errors = lambda_gamma_ridge_cv(y_train_0, tX_train_drop_invalid, lambdas, 4, 1000, 1, 1)
val_loss_per_lambda = np.mean(validation_errors, axis=0, dtype=np.float64)
np.argmin(val_loss_per_lambda)
val_loss_per_lambda[3]

0.14668964306932888

# GD least squares

In [93]:
def lambda_gamma_sgd_cv(y, tx, algorithm, lambdas, gammas, K, max_iters, batch_size, seed):
  """Do K-fold cross-validation for each value in lambdas and gammas, at every iteration.
  
  Inputs:
  y : np array
    (N, 1) or (N,)
  tx : np array
    (N, D)
  algorithm : string
    The algorithm to use for training
    Can take any value in { "LEAST_SQUARE" , "LOGISTIC_REGRESSION", "REGULARIZED_LOGISTIC_REGRESSION"}
  lambdas : iterable
    Regularisation parameters for cost function
  gamma : iterable
    Learning rates for SGD
  K : int
    Number of folds
  max_iters : int
    Maxium number of iterations for SGD
  batch_size : int
    Size of mini-batches
  seed : int
    Seed for pseudo-random number generation
  
  Outputs:
  w_best : np array
    (D, len(lambdas), len(lambdas))
    Trained weights that produced the smallest validation error
    over all folds, for each lambda and gamma
  training_errors : np array
    (K, len(lambdas), len(lambdas))
    Training loss for each fold, for each lambda and gamma
  validation_errors : np array
    (K, len(lambdas), len(lambdas))
    Validation loss for each fold, for each lambda and gamma
  """
  # loss = loss_kinds[algorithm][0]
  y, tx = prepare_dimensions(y, tx)

  N = len(y)
  len_lambdas = len(lambdas)
  len_gammas = len(gammas)

  initial_w = np.ones((tx.shape[1], 1))
  w_best = np.zeros((tx.shape[1], len_lambdas, len_gammas))

  training_errors = np.zeros((K, len_lambdas, len_gammas))
  validation_errors = np.zeros((K, len_lambdas, len_gammas))
  min_error = np.inf * np.ones((len_lambdas, len_gammas))

  k_indices = build_k_indices(y, K, seed)

  for k in range(K):
    # Take all but the k-th row of tx and y
    tx_train, y_train = map(lambda a: a[np.delete(k_indices, k).flatten()], (tx, y))
    # Take the k-th row of tx and y
    tx_test, y_test = map(lambda a: a[k_indices[k]], (tx, y))

    for i, lambda_ in enumerate(lambdas):
      for j, gamma in enumerate(gammas):
        # Train
        w, loss_tr = SGD(y_train, tx_train, initial_w, max_iters, gamma, algorithm, batch_size, lambda_)
        # Test
        # loss_te = loss(y_test, tx_test, w, lambda_)
        loss_te = compute_mse_loss_regularized(y_test, tx_test, w, 0)
        
        training_errors[k, i, j] = loss_tr
        validation_errors[k, i, j] = loss_te

        # Keep the weights that give the lowest loss_te
        if loss_te < min_error[i, j]:
          min_error[i, j] = loss_te
          w_best[:, i, j] = w.ravel()

  return w_best, training_errors, validation_errors

## Least squares SGD minibatch =1

In [172]:
gammas = np.logspace(-8, -1, 8)
lambdas = np.array([0])
w_best, training_errors, validation_errors = lambda_gamma_sgd_cv(y_train_0, tX_train_drop_invalid, "LEAST_SQUARE",
                                                                 lambdas, gammas, 4, 1000, 1, 1)
val_loss_per_lambda = np.mean(validation_errors, axis=0, dtype=np.float64).ravel()
idx_min = np.argmin(val_loss_per_lambda)
print(gammas[idx_min])
print(val_loss_per_lambda[idx_min])

0.01
0.18355890250365506


In [173]:
gammas = np.logspace(-8, -1, 8)
lambdas = np.array([0])
w_best, training_errors, validation_errors = lambda_gamma_sgd_cv(y_train_0, tX_train_replace_invalid, "LEAST_SQUARE",
                                                                 lambdas, gammas, 4, 1000, 1, 1)
val_loss_per_lambda = np.mean(validation_errors, axis=0, dtype=np.float64).ravel()
idx_min = np.argmin(val_loss_per_lambda)
print(gammas[idx_min])
print(val_loss_per_lambda[idx_min])

0.01
0.2437525676296876


## Least squares GD minibatch =N

In [112]:
gammas = np.logspace(-8, -1, 8)
lambdas = np.array([0])
w_best, training_errors, validation_errors = lambda_gamma_sgd_cv(y_train_0, tX_train_drop_invalid, "LEAST_SQUARE",
                                                                 lambdas, gammas, 4, 100, len(y_train_0), 1)
val_loss_per_lambda = np.mean(validation_errors, axis=0, dtype=np.float64).ravel()
idx_min = np.argmin(val_loss_per_lambda)
print(gammas[idx_min])
print(val_loss_per_lambda[idx_min])

0.1
0.15022318035698273


In [118]:
import pickle as pk
with open('../../pickled_files/LSGD100iters', 'wb') as file:
        pk.dump([w_best, training_errors, validation_errors], file)

In [119]:
with open('../../pickled_files/LSGD100iters', 'rb') as file:
        w_best, training_errors, validation_errors = pk.load(file)

In [148]:
gammas = np.logspace(-8, -1, 8)
lambdas = np.array([0])
w_best, training_errors, validation_errors = lambda_gamma_sgd_cv(y_train_0, tX_train_replace_invalid, "LEAST_SQUARE",
                                                                 lambdas, gammas, 4, 100, len(y_train_0), 1)
val_loss_per_lambda = np.mean(validation_errors, axis=0, dtype=np.float64).ravel()
idx_min = np.argmin(val_loss_per_lambda)
print(gammas[idx_min])
print(val_loss_per_lambda[idx_min])

0.1
0.1743628693119798


In [149]:
import pickle as pk
with open('../../pickled_files/LSGD100itersReplacedByMean', 'wb') as file:
        pk.dump([w_best, training_errors, validation_errors], file)

## Logistic regression SGD

In [174]:
gammas = np.logspace(-8, -1, 8)
lambdas = np.array([0])
w_best, training_errors, validation_errors = lambda_gamma_sgd_cv(y_train_0, tX_train_drop_invalid, "LOGISTIC_REGRESSION",
                                                                 lambdas, gammas, 4, 1000, 1, 1)
val_loss_per_lambda = np.mean(validation_errors, axis=0, dtype=np.float64).ravel()
idx_min = np.argmin(val_loss_per_lambda)
print(gammas[idx_min])
print(val_loss_per_lambda[idx_min])

0.01
1.1202573806603948


In [175]:
gammas = np.logspace(-8, -1, 8)
lambdas = np.array([0])
w_best, training_errors, validation_errors = lambda_gamma_sgd_cv(y_train_0, tX_train_replace_invalid, "LOGISTIC_REGRESSION",
                                                                 lambdas, gammas, 4, 1000, 1, 1)
val_loss_per_lambda = np.mean(validation_errors, axis=0, dtype=np.float64).ravel()
idx_min = np.argmin(val_loss_per_lambda)
print(gammas[idx_min])
print(val_loss_per_lambda[idx_min])

0.1
2.1215252308907826


## Regularized logistic regression SGD

In [176]:
gammas = np.logspace(-8, -1, 8)
lambdas = np.logspace(-8, -1, 8)
w_best, training_errors, validation_errors = lambda_gamma_sgd_cv(y_train_0, tX_train_drop_invalid, "REGULARIZED_LOGISTIC_REGRESSION",
                                                                 lambdas, gammas, 4, 1000, 1, 1)
val_loss_per_lambda_per_gamma = np.mean(validation_errors, axis=0, dtype=np.float64)
idx_min = np.unravel_index(np.argmin(val_loss_per_lambda_per_gamma), val_loss_per_lambda_per_gamma.shape)
print(lambdas[idx_min[0]])
print(gammas[idx_min[1]])
print(val_loss_per_lambda_per_gamma[idx_min])

0.1
0.01
0.323223678135669


In [177]:
gammas = np.logspace(-8, -1, 8)
lambdas = np.logspace(-8, -1, 8)
w_best, training_errors, validation_errors = lambda_gamma_sgd_cv(y_train_0, tX_train_replace_invalid, "REGULARIZED_LOGISTIC_REGRESSION",
                                                                 lambdas, gammas, 4, 1000, 1, 1)
val_loss_per_lambda_per_gamma = np.mean(validation_errors, axis=0, dtype=np.float64)
idx_min = np.unravel_index(np.argmin(val_loss_per_lambda_per_gamma), val_loss_per_lambda_per_gamma.shape)
print(lambdas[idx_min[0]])
print(gammas[idx_min[1]])
print(val_loss_per_lambda_per_gamma[idx_min])

0.1
0.01
0.47637808771066736


In [178]:
gammas = np.logspace(-8, -1, 8)
lambdas = np.logspace(-8, -1, 8)
exp_X = build_poly(tX_train_replace_invalid, 4)
w_best, training_errors, validation_errors = lambda_gamma_sgd_cv(y_train_0, exp_X, "REGULARIZED_LOGISTIC_REGRESSION",
                                                                 lambdas, gammas, 4, 1000, 1, 1)